In [ ]:
import pandas as pd
import csv
from collections import OrderedDict

MBE_table = pd.read_csv('./spreadsheets/MBINs_MBONs.csv')
new_upstream_neuron_table = pd.read_csv('./spreadsheets/FBbt_shortname.tsv', sep='\t', header=None)
existing_IDs = pd.read_csv('./spreadsheets/existing_ID.csv', sep=',')

#MBE_table[:10]
#new_upstream_neuron_table[:10]

In [ ]:
#dict of new neuron short names (keys) and FBbt IDs (values)
ID_lookup = dict(zip(new_upstream_neuron_table[0],new_upstream_neuron_table[1]))#FBbt IDs for existing terms

#add MB and group IDs
ID_lookup.update(dict(zip(existing_IDs.Keys,existing_IDs.Values)))#FBbt IDs for existing terms
#ID_lookup

In [ ]:
# Make a dictionary with key = column header & value = template specification (first row of table).

#ID column (optionally add type (default 'class') and class_type (default 'subclass') columns here)

template_seed = OrderedDict([('ID' , 'ID')])

#Synapsed_by
template_seed.update([("Synapsed_by" , "SC 'synapsed by' some % SPLIT=|")])
    
#Synapsed_to
template_seed.update([("Synapsed_to" , "SC 'synapsed to' some % SPLIT=|")])
    
# Create dataFrame for template
# from_records takes a list of dicts - one for each row.  We only have one row.

template = pd.DataFrame.from_records([template_seed])

#template

In [ ]:
count = 0 #0 = FBN-1

for i in MBE_table.index:

    r = MBE_table.short[count]
    row_od = OrderedDict([]) #new template row as an empty ordered dictionary
    for c in template.columns: #make columns and blank data for new template row
        row_od.update([(c , "")])
    
    #add ID
    row_od["ID"] = ID_lookup[r]
    
    #synapsing
    if pd.notna(MBE_table.synapsed_by[count]):
        synby_ID_list = [ID_lookup[x] for x in MBE_table.synapsed_by[count].split(" ")]
        synby_ID_string = ""
        for i in synby_ID_list:
            synby_ID_string = synby_ID_string + i + " | "
        row_od["Synapsed_by"] = synby_ID_string.rstrip(" |")
        
    if pd.notna(MBE_table.synapsed_to[count]):
        synto_ID_list = [ID_lookup[x] for x in MBE_table.synapsed_to[count].split(" ")]
        synto_ID_string = ""
        for i in synto_ID_list:
            synto_ID_string = synto_ID_string + i + " | "
        row_od["Synapsed_to"] = synto_ID_string.rstrip(" |")    
    
    #make new row into a DataFrame and add it to template
    new_row = pd.DataFrame.from_records([row_od])
    template = pd.concat([template, new_row], ignore_index=True, sort=False)

    count +=1
    
#template

In [ ]:
template.to_csv("./MB_template.tsv", sep = "\t", header=True, index=False)